<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/Seq2Seq_Paper_news_Final(nltk).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os

In [2]:
text = pd.read_excel('/content/drive/My Drive/AI NLP 알고리즘 포트폴리오/Seq2Seq/3_문어체_뉴스(1)_200226.xlsx')
text

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
0,20961563,2018-10-01 00:00:00,"경제,국제경제","IT_과학,IT_과학일반","문화,문화일반",http://www.sedaily.com/NewsView/1S5QRSR5T2,서울경제,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,20930897,2018-09-14 00:00:00,"IT_과학,IT_과학일반",NaN,NaN,http://www.sedaily.com/NewsView/1S4MSFM0IH,서울경제,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,20307875,2019-04-19 00:00:00,"IT_과학,과학",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,20314977,2019-04-27 00:00:00,"국제,중국","경제,국제경제",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,20268904,2019-03-13 00:00:00,"문화,방송_연예","사회,미디어",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.
...,...,...,...,...,...,...,...,...,...
200006,2469942,20181022,"사회,노동_복지","사회,사건_사고","사회,미디어",http://www.hani.co.kr/arti/opinion/because/866...,한겨레,당시에는 경찰의 금지통보로 청와대 근처 집회가 불가능하던 시절이라 과연 경찰이 막지...,"At the time, it was a period when it was impos..."
200007,2462113,20181019,"사회,사건_사고","정치,정치일반","사회,노동_복지",http://www.hani.co.kr/arti/society/society_gen...,한겨레,양승태 대법원과 박근혜 청와대의 대표적 재판거래 의혹 사건으로 꼽히는 전국교직원노동...,The testimony of a Blue House official at the ...
200008,2463985,20181019,"경제,산업_기업","사회,사회일반",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,윤석열 서울중앙지검장이 19일 국정감사에 출석해 ‘적폐수사 1호’로 꼽히는 한국항공...,Seoul Central District Prosecutor's Office Chi...
200009,2463107,20181019,"사회,사건_사고","국제,유럽_EU",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,대법원에 재상고된 지 5년이 넘도록 재판이 지연되면서 양승태 대법원장 시절 대표적인...,The date of the ruling of the second appeal re...


In [3]:
text = text[:70000]
text = text[['원문', '번역문']]

In [4]:
ko = text['원문']
eng = text['번역문']

In [5]:
# eng = eng.apply(lambda x : '<sos> '+ x + ' <eos>')

In [6]:
eng[0] # eg = eng[i]

"Skinner's reward is mostly eye-watering."

In [7]:
ko[0] # k = ko[i]

'스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'

In [8]:
encoder_input = [k.split() for k in ko] # 단어 단위로 encoder_input에 담는다.
decoder_input = [['<sos>'] + eg.split() for eg in eng]
decoder_target = [eg.split() + ['<eos>'] for eg in eng]

In [9]:
encoder_input[0]

['스키너가', '말한', '보상은', '대부분', '눈으로', '볼', '수', '있는', '현물이다.']

In [10]:
decoder_input[0]

['<sos>', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.']

In [11]:
decoder_target[0]

["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', '<eos>']

In [12]:
# pd_decoder_input = pd.DataFrame(decoder_input)
# pd_decoder_input = pd_decoder_input.apply(lambda x: '<sos> ' + x)
# pd_decoder_input

In [13]:
decoder_input[:5]

[['<sos>', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.'],
 ['<sos>', 'Even', 'some', 'problems', 'can', 'be', 'predicted.'],
 ['<sos>', 'Only', 'God', 'will', 'exactly', 'know', 'why.'],
 ['<sos>', 'Businesses', 'should', 'not', 'overlook', "China's", 'dispute.'],
 ['<sos>', 'Slow-beating', 'songs', 'often', 'float', 'over', 'time.']]

In [14]:
decoder_target[:5]

[["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', '<eos>'],
 ['Even', 'some', 'problems', 'can', 'be', 'predicted.', '<eos>'],
 ['Only', 'God', 'will', 'exactly', 'know', 'why.', '<eos>'],
 ['Businesses', 'should', 'not', 'overlook', "China's", 'dispute.', '<eos>'],
 ['Slow-beating', 'songs', 'often', 'float', 'over', 'time.', '<eos>']]

In [15]:
# 케라스 토크나이저를 통해 단어집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친다.

tokenizer_ko = Tokenizer(filters="", lower=False)
tokenizer_ko.fit_on_texts(encoder_input)
encoder_input_ = tokenizer_ko.texts_to_sequences(encoder_input)

tokenizer_eng = Tokenizer(filters="", lower=False)
tokenizer_eng.fit_on_texts(decoder_input)
decoder_input_ = tokenizer_eng.texts_to_sequences(decoder_input)
decoder_target_ = tokenizer_eng.texts_to_sequences(decoder_target)

In [16]:
encoder_input_ = pad_sequences(encoder_input_, padding="post")
decoder_input_ = pad_sequences(decoder_input_, padding="post")
decoder_target_ = pad_sequences(decoder_target_, padding="post")

In [17]:
decoder_target_ = pad_sequences(decoder_target_, padding='post')
decoder_target_.shape

(70000, 27)

In [18]:
encoder_input_.shape, decoder_input_.shape, decoder_target_.shape

((70000, 30), (70000, 28), (70000, 27))

In [19]:
ko_vocab_size = len(tokenizer_ko.word_index) + 1
eng_vocab_size = len(tokenizer_eng.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(ko_vocab_size, eng_vocab_size))

한국어 단어 집합의 크기 : 236187, 영어 단어 집합의 크기 : 99007


In [20]:
ko_to_index = tokenizer_ko.word_index
index_to_ko = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용

eng_to_index = tokenizer_eng.word_index # 훈련 후 예측 과정에서 사용
index_to_eng = tokenizer_eng.index_word # 훈련 후 결과 비교할 때 사용

In [21]:
eng_to_index_ = eng_to_index
eng_to_index_['<eos>'] = 22163

In [22]:
indices = np.arange(encoder_input_.shape[0])
np.random.shuffle(indices)
print(indices)

[  345  8446 11919 ... 31648 10173 58926]


In [23]:
encoder_input_ = encoder_input_[indices]
decoder_input_ = decoder_input_[indices]
decoder_target_ = decoder_target_[indices]

In [24]:
# 훈련 데이터 분리

n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [25]:
encoder_input_train = encoder_input_[:-n_of_val]
decoder_input_train = decoder_input_[:-n_of_val]
decoder_target_train = decoder_target_[:-n_of_val]

encoder_input_test = encoder_input_[-n_of_val:]
decoder_input_test = decoder_input_[-n_of_val:]
decoder_target_test = decoder_target_[-n_of_val:]

In [26]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

# 차원 25, 16, 16로 조정

(66700, 30)
(66700, 28)
(66700, 27)
(3300, 30)
(3300, 28)
(3300, 27)


In [27]:
a_zero = np.zeros((66700, 1))
b_zero = np.zeros((3300, 1))

decoder_target_train = np.append(decoder_target_train, a_zero, axis=1)
decoder_target_test = np.append(decoder_target_test, b_zero, axis=1)

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(66700, 30)
(66700, 28)
(66700, 28)
(3300, 30)
(3300, 28)
(3300, 28)


In [28]:
# a_zero = np.zeros((6700, 1))
# b_zero = np.zeros((3300, 1))

# np.append(decoder_target_train, a_zero, axis=1)
# np.append(decoder_target_test, b_zero, axis=1)

# print(decoder_target_train.shape)
# print(decoder_target_test.shape)

numpy 배열 확인 및 조정(연습)

In [29]:
decoder_input_test

array([[   2,   11,   80, ...,    0,    0,    0],
       [   2, 5056,    6, ...,    0,    0,    0],
       [   2,   57,  192, ...,    0,    0,    0],
       ...,
       [   2,  751, 1561, ...,    0,    0,    0],
       [   2,  675, 5620, ...,    0,    0,    0],
       [   2,   11, 2574, ..., 3274, 3642,    0]], dtype=int32)

In [30]:
testnp = np.arange(1, 21)
testnp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [31]:
testnp2 = testnp.reshape(5, 4)
testnp2.shape

(5, 4)

In [32]:
testnp2.shape

(5, 4)

In [33]:
a = np.zeros((5, 1))
np.append(testnp2, a, axis=1)

# 이런식으로 decoder_target_test / train을 수동으로 zeros패딩 시켜주자.

array([[ 1.,  2.,  3.,  4.,  0.],
       [ 5.,  6.,  7.,  8.,  0.],
       [ 9., 10., 11., 12.,  0.],
       [13., 14., 15., 16.,  0.],
       [17., 18., 19., 20.,  0.]])

## Seq2Seq 기계 번역기 만들기

In [34]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [35]:
latent_dim = 50

In [36]:
# 인코더

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(ko_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [37]:
# 디코더

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(eng_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [38]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [39]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [40]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     11809350    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     4950350     input_2[0][0]                    
_______________________________________________________________________________________

In [41]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test), batch_size = 128, epochs = 10)

Epoch 1/10
522/522 [==============================] - 279s 534ms/step - loss: 6.4680 - acc: 0.2423 - val_loss: 5.7699 - val_acc: 0.3047
Epoch 2/10
522/522 [==============================] - 275s 528ms/step - loss: 5.6221 - acc: 0.3183 - val_loss: 5.5184 - val_acc: 0.3286
Epoch 3/10
522/522 [==============================] - 275s 527ms/step - loss: 5.3583 - acc: 0.3419 - val_loss: 5.3085 - val_acc: 0.3489
Epoch 4/10
522/522 [==============================] - 275s 527ms/step - loss: 5.1686 - acc: 0.3570 - val_loss: 5.1718 - val_acc: 0.3605
Epoch 5/10
522/522 [==============================] - 276s 528ms/step - loss: 5.0347 - acc: 0.3674 - val_loss: 5.0911 - val_acc: 0.3674
Epoch 6/10
522/522 [==============================] - 275s 527ms/step - loss: 4.9374 - acc: 0.3740 - val_loss: 5.0301 - val_acc: 0.3717
Epoch 7/10
522/522 [==============================] - 274s 525ms/step - loss: 4.8608 - acc: 0.3793 - val_loss: 4.9963 - val_acc: 0.3741
Epoch 8/10
522/522 [============================

In [42]:
NoneType = None
type(NoneType)

NoneType

## Seq2Seq 번역기 작동시키기

In [43]:
# 인코더

encoder_model = Model(encoder_inputs, encoder_states)

In [44]:
# 디코더

# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [45]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [46]:
def decode_sequence(input_seq):

    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = eng_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정한다.

    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_eng[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [47]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2ko(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_ko[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2eng(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=eng_to_index['<sos>']) and i!=eng_to_index['<eos>']):
            temp = temp + index_to_eng[i] + ' '
    return temp

In [111]:
for seq_index in [3,25,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2ko(encoder_input_train[seq_index]))
  print("번역문 :",seq2eng(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence)
  print("\n")

원문 :  넷째로는 주입식 교육 대신 학생들이 수업에 주도적으로 이끌어가는 ‘참여형 교육’을 위해 교과서 내용도 큰 폭으로 바뀌었다. 
번역문 : Fourthly, textbook contents have changed drastically for "participatory education", in which students take the initiative in class instead of learning by rote. 
예측문 :  Unlike the existing market, the world's largest number


원문 :  푸른색 톤의 화면에 다채로운 색점들이 별처럼 빛나거나 종이 위에 푸른색의 선염이 번지는 김환기 스타일의 서정적 화면이다. 
번역문 : It's a lyrical screen of Kim Hwan-ki's style, with colorful colors shining like stars on a blue-toned screen or spreading blue-colored ombre on paper. 
예측문 :  The number of the world's largest number of the first


원문 :  생태적 먹거리, 쉴거리, 볼거리를 찾겠다는 목표 아래 15년 계획의 친환경 수련원 조성 사업이 진행되고 있다. 
번역문 : Under the goal of finding ecological food, rest and sights, a 15-year program to create an eco-friendly training center is under way. 
예측문 :  According to the U.S. and the U.S. and the U.S. and


원문 :  문재인 대통령과 이재용 삼성전자 부회장이 9일 저녁(한국 시각) 예정인 ‘인도 만남’을 놓고 여러 해석이 나온다. 
번역문 : President Moon Jae-in and Vice-Chairman Lee Jae-y

## BLEU 사용(nltk 말고, 직접 구현으로)

In [49]:
from collections import Counter
from nltk.util import ngrams

In [50]:
# 단순 카운트 함수
def simple_count(tokens, n): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운트

In [53]:
def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram 카운트
    
    temp = dict()

    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

In [55]:
def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

짧은 문장도 있기 때문에 감점을 주어야 한다.

In [101]:
def closest_ref_length(candidate, reference_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

In [102]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

In [103]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

## nltk를 이용한 BLEU(훨씬 편하고 모듈화)

In [109]:
import nltk.translate.bleu_score as bleu

candidate = 'In China, more than 10,000 students a year go into post-doctoral courses.'
references = 'The government has been to be held in the first time'
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

0.537284965911771


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [112]:
candidate2 = 'President Moon Jae-in and Vice-Chairman Lee Jae-yong come up with various interpretations on "India Meeting" scheduled for the evening of the 9th (Korean time)'
references2 = 'The U.S. President Trump said that President Trump'
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

0.537284965911771


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
